In [35]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os

In [7]:
# Geting csv files
list_of_files = os.listdir("./")
csv_files = []

for _file in list_of_files:
    if _file.endswith(".csv"):
        csv_files.append(_file)

csv_files

['5_When_rebuilding.csv',
 '1_After_reconstruction.csv',
 '1_When_rebuilding.csv',
 '2_Before_renovation.csv',
 '6_After_reconstruction.csv',
 '2_When_rebuilding.csv',
 '4_Before_renovation.csv',
 '5_After_reconstruction.csv',
 '3_When_rebuilding.csv',
 '3_After_reconstruction.csv',
 '6_Before_renovation.csv',
 '4_After_reconstruction.csv',
 '1_Before_renovation.csv',
 '3_Before_renovation.csv',
 '6_When_rebuilding.csv',
 '2_After_reconstruction.csv',
 '5_Before_renovation.csv',
 '4_When_rebuilding.csv']

In [87]:
def custom_slicer(signal, fs=825.8):
    '''
    Returns: first 10 seconds
             last 30 seconds of the signal
    '''

    return signal[:int(fs * 10)].reshape(int(fs * 10), 1), signal[int(-30 * fs):].reshape(int(30 * fs), 1)

In [88]:
def slicer(signal, fs=825.8, chunk_size_seconds=10):
    '''
    Returns: sliced signal in desired seconds chunks
    Note: size of the signal should be a true nominator of chunk size
    '''
    chunk_size = int(fs * chunk_size_seconds)
    return [signal[_ * chunk_size : (_+1)*chunk_size ].reshape(int(10 * fs), 1) for _ in range(int(len(signal)/chunk_size))]

In [78]:
def signal_to_heatmap(signal):
    '''
    Returns the heatmap of signal
    '''
    # Desired size in pixels
    width_px = 600
    height_px = 600

    # DPI (dots per inch)
    dpi = 300

    # Convert to inches
    width_in = width_px / dpi
    height_in = height_px / dpi

    # Set the size of the heatmap
    plt.figure(figsize=(width_in, height_in), dpi=dpi)
    
    sns.heatmap(signal, xticklabels=False, yticklabels=False, cbar=False)

In [59]:
def dir_creator(csv_file_name, columns):
    '''
    Creating directoies for each column of csv files
        to save heatmaps into them
    '''
    file_name = csv_file_name.split('.')[0]
    os.mkdir(file_name)
    for _ in range(columns):
        os.mkdir(f"{file_name}/{_}")
        os.mkdir(f"{file_name}/{_}/train_out")
        os.mkdir(f"{file_name}/{_}/train_in")

In [90]:
def save_heatmap(csv_file_name, column, counter, train_status):
    '''
    Saving plots in png format in their specific path
    '''
    path = f"{csv_file_name.split('.')[0]}/{column}/{train_status}/{csv_file_name}_{column}_{counter}.png"
    plt.savefig(path , dpi=300, bbox_inches='tight')

In [ ]:
def remove_five_zeros(arr):
    """Remove sequences of 5 successive zeros from an array."""
    result = []
    zero_count = 0

    for i in range(len(arr)):
        if arr[i] == 0:
            zero_count += 1
        else:
            if zero_count == 5:
                result = result[:-5]  # Remove the last 5 zeros from the result
            zero_count = 0
            result.append(arr[i])
        
        # If we've reached the end and have exactly 5 zeros at the end
        if i == len(arr) - 1 and zero_count == 5:
            result = result[:-5]
    
    return np.array(result)

In [ ]:
# Initializing fs
fs=825.8

In [ ]:
# Reading and processing csv files
for csv_file in csv_files:
    # Reading files
    df = pd.read_csv(csv_file)

    # Nan to zero
    df = df.fillna(0)
    
    # Loading each column of df into dict
    columns_dict = {}
    for col in df.columns:
        columns_dict[col] = df[col].to_numpy()
    # Removing first incremental column
    columns_dict.pop('Unnamed: 0') 

    # Creating dirs
    dir_creator(csv_file, len(columns_dict.keys()))

    heatmaps_counter = 1

    for col in columns_dict.keys():
        # Removing 5 successive zeros
        columns_dict[col] = remove_five_zeros(columns_dict[col])

        # handling train out part
        first_10, last_30 = custom_slicer(columns_dict[col], fs)
        last_30_signals_chunked = slicer(last_30)

        signal_to_heatmap(first_10)
        save_heatmap(csv_file, col, heatmaps_counter, "train_out")
        heatmaps_counter += 1
        
        for sig_chunk in last_30_signals_chunked:
            signal_to_heatmap(sig_chunk)
            save_heatmap(csv_file, col, heatmaps_counter, "train_out")
            heatmaps_counter += 1
        
        # handling train in part
        train_in_sig_chunks = slicer(columns_dict[col][10:-30])
        for sig_chunk in train_in_sig_chunks:
            signal_to_heatmap(sig_chunk)
            save_heatmap(csv_file, col, heatmaps_counter, "train_in")
            heatmaps_counter += 1